https://github.com/kpu/kenlm
Old: http://victor.chahuneau.fr/notes/2012/07/03/kenlm.html
http://smithamilli.com/blog/kneser-ney/

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/stephanie/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
from article import Library
path = '/Users/stephanie/data/newsela_article_corpus_2016-01-29/'
library = Library(path)
library.create_grade_level_library()

In [18]:
library.grade_level_tokenized_sentences[12]

['<s>',
 'WASHINGTON',
 'An',
 'abolitionist',
 '</s>',
 '<s>',
 'The',
 'longest-serving',
 'first',
 'lady',
 '</s>',
 '<s>',
 'The',
 'Labor',
 'secretary',
 'through',
 'the',
 'Great',
 'Depression',
 '</s>',
 '<s>',
 'The',
 'founder',
 'of',
 'the',
 'Girl',
 'Scouts',
 '</s>',
 '<s>',
 'These',
 'are',
 'some',
 'of',
 'the',
 'candidates',
 'to',
 'be',
 'the',
 'first',
 'woman',
 'on',
 'U',
 '.',
 'S',
 '.',
 'currency',
 'notes',
 'in',
 'more',
 'than',
 'a',
 'century',
 '</s>',
 '<s>',
 'Treasury',
 'Secretary',
 'Jacob',
 'J',
 '.',
 'Lew',
 'announced',
 'the',
 'plans',
 'this',
 'week',
 'saying',
 'the',
 'all-male',
 'lineup',
 'on',
 'American',
 'money',
 'has',
 'gone',
 'on',
 'long',
 'enough',
 '</s>',
 '<s>',
 'We',
 'will',
 'right',
 'that',
 'wrong',
 'and',
 'when',
 'the',
 'new',
 'redesigned',
 '10',
 'note',
 'is',
 'released',
 'it',
 'will',
 'bear',
 'the',
 'portrait',
 'of',
 'a',
 'woman',
 'he',
 'said',
 'at',
 'the',
 'National',
 'Archives

In [10]:
!ls ~/github/kenlm/python

_kenlm.pxd example.py kenlm.cpp  kenlm.pyx


In [14]:
#!/usr/bin/env python
import os
import kenlm

# LM = os.path.join(os.path.dirname(__file__), '..', 'lm', 'test.arpa')
LM = '/Users/stephanie/github/kenlm/lm/test.arpa'
print(LM)
model = kenlm.LanguageModel(LM)
print('{0}-gram model'.format(model.order))

sentence = 'language modeling is fun .'
print(sentence)
print(model.score(sentence))

# Check that total full score = direct score
def score(s):
    return sum(prob for prob, _, _ in model.full_scores(s))

assert (abs(score(sentence) - model.score(sentence)) < 1e-3)

# Show scores and n-gram matches
words = ['<s>'] + sentence.split() + ['</s>']
for i, (prob, length, oov) in enumerate(model.full_scores(sentence)):
    print('{0} {1}: {2}'.format(prob, length, ' '.join(words[i+2-length:i+2])))
    if oov:
        print('\t"{0}" is an OOV'.format(words[i+1]))

# Find out-of-vocabulary words
for w in words:
    if not w in model:
        print('"{0}" is an OOV'.format(w))

#Stateful query
state = kenlm.State()
state2 = kenlm.State()
#Use <s> as context.  If you don't want <s>, use model.NullContextWrite(state).
model.BeginSentenceWrite(state)
accum = 0.0
accum += model.BaseScore(state, "a", state2)
accum += model.BaseScore(state2, "sentence", state)
#score defaults to bos = True and eos = True.  Here we'll check without the end
#of sentence marker.
assert (abs(accum - model.score("a sentence", eos = False)) < 1e-3)
accum += model.BaseScore(state, "</s>", state2)
assert (abs(accum - model.score("a sentence")) < 1e-3)

/Users/stephanie/github/kenlm/lm/test.arpa
5-gram model
language modeling is fun .
-64.59443664550781
-2.4106082916259766 1: language
	"language" is an OOV
-15.0 2: language modeling
	"modeling" is an OOV
-23.6878719329834 1: is
-2.2966649532318115 1: fun
	"fun" is an OOV
-21.139057159423828 1: .
-0.060235898941755295 2: . </s>
"language" is an OOV
"modeling" is an OOV
"fun" is an OOV


In [ ]:
#!/usr/bin/env python
import os
import kenlm

# LM = os.path.join(os.path.dirname(__file__), '..', 'lm', 'test.arpa')
LM = '/Users/stephanie/github/kenlm/lm/test.arpa'
print(LM)
model = kenlm.LanguageModel(LM)
print('{0}-gram model'.format(model.order))

sentence = 'language modeling is fun .'
print(sentence)
print(model.score(sentence))

# Check that total full score = direct score
def score(s):
    return sum(prob for prob, _, _ in model.full_scores(s))

assert (abs(score(sentence) - model.score(sentence)) < 1e-3)

# Show scores and n-gram matches
words = ['<s>'] + sentence.split() + ['</s>']
for i, (prob, length, oov) in enumerate(model.full_scores(sentence)):
    print('{0} {1}: {2}'.format(prob, length, ' '.join(words[i+2-length:i+2])))
    if oov:
        print('\t"{0}" is an OOV'.format(words[i+1]))

# Find out-of-vocabulary words
for w in words:
    if not w in model:
        print('"{0}" is an OOV'.format(w))

#Stateful query
state = kenlm.State()
state2 = kenlm.State()
#Use <s> as context.  If you don't want <s>, use model.NullContextWrite(state).
model.BeginSentenceWrite(state)
accum = 0.0
accum += model.BaseScore(state, "a", state2)
accum += model.BaseScore(state2, "sentence", state)
#score defaults to bos = True and eos = True.  Here we'll check without the end
#of sentence marker.
assert (abs(accum - model.score("a sentence", eos = False)) < 1e-3)
accum += model.BaseScore(state, "</s>", state2)
assert (abs(accum - model.score("a sentence")) < 1e-3)

In [ ]:
import sys
import nltk

for line in sys.stdin:
    for sentence in nltk.sent_tokenize(line):
        print(' '.join(nltk.word_tokenize(sentence)).lower())

In [29]:
#!/usr/bin/env python

import sys
import nltk
import pandas as pd
from collections import defaultdict
import os
import kenlm


def get_article_text(path, filename):
    """
    Returns
    :return:
    """
    txt = open(path + 'articles/' + filename, 'r')
    return txt.read()

# path = '/Users/stzeng/code/github/autograder/data/newsela_article_corpus_2016-01-29/'
path = '/Users/stephanie/data/newsela_article_corpus_2016-01-29/'
metadata = pd.read_csv(path + 'articles_metadata.csv')
metadata.loc[:, 'grade_level'] = metadata.grade_level.astype('int')

d = defaultdict()

for row in metadata.itertuples():

    if row.grade_level == 8:
        text = get_article_text(path, row.filename)

        for sentence in nltk.sent_tokenize(text):
            if not d.get(8):
                d[8] = ' '.join(nltk.word_tokenize(sentence)).lower() + '\n'
            else:
                d[8] = d[8] + ' '.join(nltk.word_tokenize(sentence)).lower() + '\n'




# LM = os.path.join(os.path.dirname(__file__), '..', 'lm', 'test.arpa')
LM = '/Users/stephanie/github/kenlm/lm/test.arpa'
print(LM)
model = kenlm.LanguageModel(LM)
print('{0}-gram model'.format(model.order))

sentence = d[8]
# print(sentence)
# sentence = "It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, it was the season of Light, it was the season of Darkness, it was the spring of hope, it was the winter of despair, we had everything before us, we had nothing before us, we were all going direct to Heaven, we were all going direct the other way—in short, the period was so far like the present period, that some of its noisiest authorities insisted on its being received, for good or for evil, in the superlative degree of comparison only."
# sentence = ' '.join(nltk.word_tokenize(sentence)).lower()
print(model.score(sentence))

# Check that total full score = direct score
def score(s):
    return sum(prob for prob, _, _ in model.full_scores(s))

assert (abs(score(sentence) - model.score(sentence)) < 1e-3)

# Show scores and n-gram matches
# words = ['<s>'] + sentence.split() + ['</s>']
# for i, (prob, length, oov) in enumerate(model.full_scores(sentence)):
#     print('{0} {1}: {2}'.format(prob, length, ' '.join(words[i+2-length:i+2])))
#     if oov:
#         print('\t"{0}" is an OOV'.format(words[i+1]))
#
# # Find out-of-vocabulary words
# for w in words:
#     if not w in model:
#         print('"{0}" is an OOV'.format(w))

# #Stateful query
# state = kenlm.State()
# state2 = kenlm.State()
# #Use <s> as context.  If you don't want <s>, use model.NullContextWrite(state).
# model.BeginSentenceWrite(state)
# accum = 0.0
# accum += model.BaseScore(state, "a", state2)
# accum += model.BaseScore(state2, "sentence", state)
# #score defaults to bos = True and eos = True.  Here we'll check without the end
# #of sentence marker.
# assert (abs(accum - model.score("a sentence", eos = False)) < 1e-3)
# accum += model.BaseScore(state, "</s>", state2)
# assert (abs(accum - model.score("a sentence")) < 1e-3)

KeyboardInterrupt: 